# 2. Grouping and Aggregating with Multiple Columns

### Objectives

+ Use multiple grouping columns
+ Aggregate multiple columns
+ Use multiple aggregating functions
+ Rename columns after grouping
+ Be aware of alternate syntax for grouping

### Overview
In this notebook we will learn how to form groups using more than one column. We will also aggregate more than one column as well as learn how to apply more than one aggregation function to each group.

## Adding Years of Experience to City of Houston Data
Before we get started with grouping and aggregating multiple columns, let's read in the City of Houston employee dataset and append a column for the years of experience.

In [1]:
import pandas as pd
emp = pd.read_csv('../data/employee.csv', parse_dates=['hire_date'])
emp.head()

,title,dept,salary,race,gender,hire_date
0,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Male,2015-02-03
1,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Male,1982-02-08
2,SENIOR POLICE OFFICER,Houston Police Department-HPD,66614.0,Black,Male,1984-11-26
3,ENGINEER,Public Works & Engineering-PWE,71680.0,Asian,Male,2012-03-26
4,CARPENTER,Houston Airport System (HAS),42390.0,White,Male,2013-11-04


### Calculate years of experience from hire date
The data was pulled on December 1, 2016. Let's use the **`dt`** accessor with the **`year`** attribute to get the year that each employee was hired. We can subtract this year from 2016 to approximate the years of experience and assign it as a new column.

In [7]:
emp['experience'] = 2016 - emp['hire_date'].dt.year

### Take a peak at distribution of experience
Use the **`value_counts`** to get a quick understanding of how experience is distributed.

In [31]:
emp['experience'].value_counts(normalize=True).head(10)

1     0.069570
9     0.053237
2     0.047792
8     0.047792
3     0.044162
10    0.041742
23    0.039322
0     0.038717
13    0.036298
7     0.036298
Name: experience, dtype: float64

In [12]:
emp.head()

,title,dept,salary,race,gender,hire_date,experience
0,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Male,2015-02-03,1
1,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Male,1982-02-08,34
2,SENIOR POLICE OFFICER,Houston Police Department-HPD,66614.0,Black,Male,1984-11-26,32
3,ENGINEER,Public Works & Engineering-PWE,71680.0,Asian,Male,2012-03-26,4
4,CARPENTER,Houston Airport System (HAS),42390.0,White,Male,2013-11-04,3


## Review grouping and aggregating with a single column
In the previous notebook, we had a single grouping column, aggregating column, and aggregating function. The following syntax was used as a guide:

**```
df.groupby('<grouping column>').agg({'<aggregating column>':'<aggregating function>'})
```**

Let's see this again by calculating the average years of experience for each gender.

In [13]:
emp.groupby('gender').agg({'experience': 'mean'})

,experience
gender,
Female,12.910588
Male,15.015472


# Grouping with Multiple Columns (use a list)
To create groups based on distinct values from multiple columns, we will need to pass a list of these columns to the **`groupby`** method. Let's find the average years of experience for every unique combination of race and gender.

In [15]:
emp.groupby(['race', 'gender']).agg({'experience': 'mean'}).reset_index()

,race,gender,experience
0,Asian,Female,16.277778
1,Asian,Male,13.500000
2,Black,Female,13.555556
3,Black,Male,13.276074
4,Hispanic,Female,10.861386
5,Hispanic,Male,12.843537
6,Native American,Female,15.750000
7,Native American,Male,15.750000
8,White,Female,14.179487
9,White,Male,17.867816


### What happened to our index?
Both race and gender are not columns and have been pushed into the index. This is called a **multi-level index** and technically a **`MultiIndex`** object. **`race`** and **`gender`** are considered **levels** of the index. They are NOT columns. You'll notice that duplicated values do not repeat in an index when they immediately follow one another.

### The MultiIndex is confusing and not necessary for beginners
In my opinion, this multi-level index only adds to confusion. By default, all grouping columns will be added to the index. From this point on, we will chain the **`reset_index`** method to return these levels to columns.

In [16]:
emp.groupby(['race', 'gender']).agg({'experience': 'mean'}).reset_index()

,race,gender,experience
0,Asian,Female,16.277778
1,Asian,Male,13.500000
2,Black,Female,13.555556
3,Black,Male,13.276074
4,Hispanic,Female,10.861386
5,Hispanic,Male,12.843537
6,Native American,Female,15.750000
7,Native American,Male,15.750000
8,White,Female,14.179487
9,White,Male,17.867816


### Isn't it easier to read with a MultiIndex?
The MultiIndex can make the results easier to read, but it makes further data analysis more difficult as you need to become familiar with special syntax just for the MultiIndex. This added complexity for beginners is not worth the benefit.

# Aggregating Multiple Columns
To aggregate multiple columns, add the column name to the dictionary paired with its aggregation function. The aggregation functions can be different. The following finds the average salary and max years of experience for each gender.

In [17]:
emp.groupby('gender').agg({'salary': 'mean', 'experience': 'max'}).reset_index()

,gender,salary,experience
0,Female,50815.556650,41
1,Male,57966.976419,58


# Grouping and Aggregating with Multiple Columns
We can combine the last two approaches to group with multiple columns along with multiple aggregating columns.

The following finds the mean salary and max experience for every unique combination of race and gender. It might make things more readable by placing the each aggregating column on a separate line.

In [18]:
emp.groupby(['race', 'gender']).agg({'salary': 'mean', 
                                     'experience': 'max'}).reset_index()

,race,gender,salary,experience
0,Asian,Female,58304.222222,35
1,Asian,Male,60622.956522,39
2,Black,Female,48133.381643,37
3,Black,Male,51853.000000,48
4,Hispanic,Female,44216.960000,37
5,Hispanic,Male,55493.064057,41
6,Native American,Female,58844.333333,21
7,Native American,Male,68850.500000,25
8,White,Female,66415.527778,41
9,White,Male,63439.195745,58


# Multiple Aggregation Functions
Let's say we want to find the min, max, mean, and median salary for each race. We do this by using a list of aggregating functions as the key in our **`agg`** dictionary.

In [19]:
emp.groupby('race').agg({'salary': ['min', 'max', 'mean', 'median']})

salary                                 
                     min       max          mean   median
race                                                     
Asian            26125.0  163228.0  60143.218391  55461.0
Black            24960.0  186192.0  50366.588803  45926.0
Hispanic         26104.0  165216.0  52533.456693  51194.0
Native American  49379.0   81239.0  64562.142857  60347.0
White            26125.0  210588.0  63834.575646  62540.0

## What's up with those column names???
The column names probably look pretty bizarre to you. Although it doesn't take much effort to decipher what each column means, the column names are not particularly friendly to work with.

Pandas created a **multi-level column index** with two levels. These are difficult to work with. There isn't a standard way to deal with them like we did with the multi-level index from above.

## Renaming all the columns
I recommend renaming all the columns after the aggregation. This is straightforward, but tedious. Assign the DataFrame's **`columns`** attribute to a list of desired column names. The list must be the same length as the original.

In [20]:
race_salary = emp.groupby('race').agg({'salary': ['min', 'max', 'mean', 'median']}).reset_index()
race_salary.columns = ['race', 'min salary', 'max salary', 'mean salary', 'median salary']
race_salary

,race,min salary,max salary,mean salary,median salary
0,Asian,26125.0,163228.0,60143.218391,55461.0
1,Black,24960.0,186192.0,50366.588803,45926.0
2,Hispanic,26104.0,165216.0,52533.456693,51194.0
3,Native American,49379.0,81239.0,64562.142857,60347.0
4,White,26125.0,210588.0,63834.575646,62540.0


If you are not planning on using the returned DataFrame then you don't need to bother renaming the columns, but having a single level index is going to be much easier to work with than a MultiIndex when you are first beginning your Pandas journey. 

## No added functionality of a MultiIndex
I actually don't think the MultiIndex offers much benefit. All data analysis is possible without it. There are some cool tricks you can do with it, but overall it will not prevent you from achieving any kind of analysis if you do not use it.

# Multiple Grouping Columns, Aggregating Columns, and Aggregating Functions
You can make complex aggregations by having multiple grouping columns, aggregating columns, and aggregating functions.

In [21]:
rg_sal_exp = emp.groupby(['race', 'gender']) \
                .agg({'salary': ['min', 'max', 'mean', 'median'],
                      'experience': ['max', 'std']}).reset_index()
rg_sal_exp

race  gender   salary                                   \
                                min       max          mean   median   
0            Asian  Female  26125.0   95950.0  58304.222222  51514.5   
1            Asian    Male  27914.0  163228.0  60622.956522  55461.0   
2            Black  Female  24960.0  150416.0  48133.381643  40581.0   
3            Black    Male  26125.0  186192.0  51853.000000  49150.0   
4         Hispanic  Female  26125.0   96157.0  44216.960000  42837.5   
5         Hispanic    Male  26104.0  165216.0  55493.064057  55437.0   
6  Native American  Female  49379.0   68299.0  58844.333333  58855.0   
7  Native American    Male  55461.0   81239.0  68850.500000  69351.0   
8            White  Female  30888.0  178331.0  66415.527778  62783.0   
9            White    Male  26125.0  210588.0  63439.195745  62540.0   

  experience             
         max        std  
0         35   9.868581  
1         39   9.645529  
2         37   9.352362  
3         48  10.290123  
4         37   9.088487  
5         41   9.859907  
6         21   4.112988  
7         25   8.098354  
8         41  10.809740  
9         58  11.783384

Again, I suggest renaming the columns for easier data manipulation.

In [22]:
rg_sal_exp.columns = ['race', 'gender', 'min salary', 'max salary', 'mean salary',
                      'median salary', 'max exp', 'std exp']
rg_sal_exp

,race,gender,min salary,max salary,mean salary,median salary,max exp,std exp
0,Asian,Female,26125.0,95950.0,58304.222222,51514.5,35,9.868581
1,Asian,Male,27914.0,163228.0,60622.956522,55461.0,39,9.645529
2,Black,Female,24960.0,150416.0,48133.381643,40581.0,37,9.352362
3,Black,Male,26125.0,186192.0,51853.000000,49150.0,48,10.290123
4,Hispanic,Female,26125.0,96157.0,44216.960000,42837.5,37,9.088487
5,Hispanic,Male,26104.0,165216.0,55493.064057,55437.0,41,9.859907
6,Native American,Female,49379.0,68299.0,58844.333333,58855.0,21,4.112988
7,Native American,Male,55461.0,81239.0,68850.500000,69351.0,25,8.098354
8,White,Female,30888.0,178331.0,66415.527778,62783.0,41,10.809740
9,White,Male,26125.0,210588.0,63439.195745,62540.0,58,11.783384


# Getting the size of each group
Let's say we just want to know the number of rows in each group. The correct aggregation function is **`size`** and not **`count`** (this returns the number of non-missing values).

In [23]:
emp.groupby(['race', 'gender']).agg({'salary': 'size'}).reset_index()

,race,gender,salary
0,Asian,Female,18
1,Asian,Male,70
2,Black,Female,216
3,Black,Male,326
4,Hispanic,Female,101
5,Hispanic,Male,294
6,Native American,Female,4
7,Native American,Male,4
8,White,Female,78
9,White,Male,522


### The aggregating column doesn't matter
The same result will be returned regardless of what aggregating column we use since the size only depends on the number of rows and not on the actual values in the column. Using the department column does not change the output.

In [24]:
emp.groupby(['race', 'gender']).agg({'dept': 'size'}).reset_index()

,race,gender,dept
0,Asian,Female,18
1,Asian,Male,70
2,Black,Female,216
3,Black,Male,326
4,Hispanic,Female,101
5,Hispanic,Male,294
6,Native American,Female,4
7,Native American,Male,4
8,White,Female,78
9,White,Male,522


## Alternative Syntax for size
You can call the **`size`** method directly after grouping. This will return the same data as a Series.

In [25]:
emp.groupby(['race', 'gender']).size()

race             gender
Asian            Female     18
                 Male       70
Black            Female    216
                 Male      326
Hispanic         Female    101
                 Male      294
Native American  Female      4
                 Male        4
White            Female     78
                 Male      522
dtype: int64

## Rename the column when using `reset_index`
When calling `reset_index` on a Series, like we did above, the new column name for the Series values will be the `name` attribute of the Series. If it doesn't exist (like in the example above) then you can supply the column name with the `name` parameter with `reset_index`.

In [28]:
emp.groupby(['race', 'gender']).size().reset_index(name='size')

,race,gender,size
0,Asian,Female,18
1,Asian,Male,70
2,Black,Female,216
3,Black,Male,326
4,Hispanic,Female,101
5,Hispanic,Male,294
6,Native American,Female,4
7,Native American,Male,4
8,White,Female,78
9,White,Male,522


# Exercises

### Problem 1
<span  style="color:green; font-size:16px">For each department and gender find the number of unique position titles, the total number of employees and the average salary. Make sure there is no multi-index for the index or columns.</span>

In [53]:
emp1 = emp.groupby(['dept', 'gender']).agg({'title':['nunique','size'], 'salary':'mean'})
emp1.columns = ['unique position', '# employee', 'salary avg']
emp1.reset_index()

,dept,gender,unique position,# employee,salary avg
0,Health & Human Services,Female,47,82,48661.961538
1,Health & Human Services,Male,22,26,59240.000000
2,Houston Airport System (HAS),Female,24,36,53174.194444
3,Houston Airport System (HAS),Male,37,70,54358.171429
4,Houston Fire Department (HFD),Female,13,21,52853.047619
5,Houston Fire Department (HFD),Male,26,363,59930.564470
6,Houston Police Department-HPD,Female,38,155,52219.925170
7,Houston Police Department-HPD,Male,27,483,63032.841121
8,Parks & Recreation,Female,15,23,40361.055556
9,Parks & Recreation,Male,20,51,38396.243243


### Problem 2
<span  style="color:green; font-size:16px">For each department, race and gender find the maximum years of experience and salary.</span>

In [38]:
emp.groupby(['dept', 'race', 'gender']).agg({'experience':'max', 'salary':'max'}).reset_index()

,dept,race,gender,experience,salary
0,Health & Human Services,Asian,Female,23,94149.0
1,Health & Human Services,Asian,Male,25,70864.0
2,Health & Human Services,Black,Female,34,103270.0
3,Health & Human Services,Black,Male,29,180416.0
4,Health & Human Services,Hispanic,Female,25,65589.0
5,Health & Human Services,Hispanic,Male,14,58406.0
6,Health & Human Services,Native American,Female,17,58855.0
7,Health & Human Services,White,Female,33,100791.0
8,Health & Human Services,White,Male,8,120799.0
9,Houston Airport System (HAS),Asian,Female,23,32157.0


## Use the college dataset for the rest of the problems

In [39]:
college = pd.read_csv('../data/college.csv')
college.head()

,instnm,city,stabbr,hbcu,menonly,womenonly,relaffil,satvrmid,satmtmid,distanceonly,...,ugds_2mor,ugds_nra,ugds_unkn,pptug_ef,curroper,pctpell,pctfloan,ug25abv,md_earn_wne_p10,grad_debt_mdn_supp
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


### Problem 3
<span  style="color:green; font-size:16px">Which city name appears the most frequently. Do this in two different ways. Do it once with and once without the `groupby` method?</span>

In [47]:
college.groupby('city').agg({'hbcu':'size'}).head()

,hbcu
city,
ARTESIA,1
Aberdeen,3
Abilene,5
Abingdon,2
Abington,1


In [54]:
col = college.groupby('city').size()
col.reset_index(name='Size').head()

,city,Size
0,ARTESIA,1
1,Aberdeen,3
2,Abilene,5
3,Abingdon,2
4,Abington,1


In [59]:
# another way to do it would be
college['city'].value_counts().head()

New York       87
Chicago        78
Houston        72
Los Angeles    56
Miami          51
Name: city, dtype: int64

### Problem 4
<span  style="color:green; font-size:16px">Does the city **`Houston`** only appear in the state of **`Texas`**?</span>

In [79]:
states = college.groupby('city').agg({'stabbr':'nunique'})
states.loc['Houston']

stabbr    2
Name: Houston, dtype: int64

In [77]:
filt = college['city'] == 'Houston'
college.loc[filt,'stabbr'].unique()

array(['TX', 'MO'], dtype=object)

In [75]:
college.loc[filt,'stabbr'].value_counts()

TX    71
MO     1
Name: stabbr, dtype: int64

### Problem 5
<span  style="color:green; font-size:16px">Find the maximum undergraduate population for each state?</span>

In [93]:
under = college.groupby('stabbr').agg({'ugds':'max'})
under.head()

,ugds
stabbr,
AK,12865.0
AL,29851.0
AR,21405.0
AS,1276.0
AZ,151558.0


### Problem 6
<span  style="color:green; font-size:16px">Among colleges that have the largest undergrad population for each state, what is the difference between the most and least populous college?</span>

In [95]:
under.sort_values('ugds', ascending=False).head()

,ugds
stabbr,
AZ,151558.0
IN,77657.0
FL,61470.0
TX,59920.0
VA,49340.0


In [97]:
under.max() - under.min()

ugds    150956.0
dtype: float64

### Problem 7: Advanced
<span  style="color:green; font-size:16px">Find the name and population of the largest college per state.</span>

In [103]:
pop = college.groupby('stabbr').agg({'ugds':'idxmax'})
pop.head()

,ugds
stabbr,
AK,60
AL,5
AR,137
AS,4138
AZ,7116


In [107]:
locs = pop['ugds']
locs

stabbr
AK      60
AL       5
AR     137
AS    4138
AZ    7116
CA    1299
CO     574
CT     641
DC     701
DE     691
FL     793
FM    4214
GA     909
GU    4140
HI     952
IA    4318
ID     979
IL    1049
IN    1189
KS    1363
KY    1424
LA    1486
MA    1690
MD    1589
ME    1542
MH    4561
MI    1818
MN    1906
MO    2061
MP    4141
MS    1978
MT    2127
NC    2726
ND    2788
NE    2159
NH    2197
NJ    2284
NM    2304
NV    2172
NY    2615
OH    2943
OK    3040
OR    3116
PA    3304
PR    4172
PW    4215
RI    3410
SC    3458
SD    3497
TN    3592
TX    3711
UT    5140
VA    3880
VI    4216
VT    3849
WA    4008
WI    4123
WV    5817
WY    4137
Name: ugds, dtype: int64

### Problem 8
<span  style="color:green; font-size:16px">Do distance only schools tend to have more or less student population than non-distance-only schools?</span>

In [110]:
college.groupby('distanceonly').agg({'ugds':'mean'})

,ugds
distanceonly,
0.0,2334.648135
1.0,6245.743590


### Problem 9
<span  style="color:green; font-size:16px">Do distance only schools tend to be more or less religously affiliated than non-distance-only schools?</span>

In [116]:
college.groupby('distanceonly').agg({'relaffil':'mean'})

,relaffil
distanceonly,
0.0,0.149635
1.0,0.050000


### Problem 10
<span  style="color:green; font-size:16px">What state has the lowest percentage of currently operating schools of those that have religious affiliation?</span>

In [127]:
filt = college['relaffil'] == 1
coll = college[filt]
col_mean = coll.groupby('stabbr').agg({'curroper':'mean'})
col_mean.head()

,curroper
stabbr,
AK,1.000000
AL,0.916667
AR,0.944444
AZ,0.444444
CA,0.585366


In [128]:
col_mean.sort_values('curroper').head()

,curroper
stabbr,
UT,0.400000
AZ,0.444444
NV,0.500000
CA,0.585366
CT,0.647059


### Problem 11
<span  style="color:green; font-size:16px">Trim the **`college`** DataFrame to only the 'race' columns - those beginning with **`ugds_`**. Create a new column called **`ugds_other`** that is the sum of any race column that averages under 4% for the entire dataset.</span>

In [129]:
college

,instnm,city,stabbr,hbcu,menonly,womenonly,relaffil,satvrmid,satmtmid,distanceonly,...,ugds_2mor,ugds_nra,ugds_unkn,pptug_ef,curroper,pctpell,pctfloan,ug25abv,md_earn_wne_p10,grad_debt_mdn_supp
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5
5,The University of Alabama,Tuscaloosa,AL,0.0,0.0,0.0,0,555.0,565.0,0.0,...,0.0261,0.0268,0.0026,0.0844,1,0.2040,0.4010,0.0853,41900,23750
6,Central Alabama Community College,Alexander City,AL,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0000,0.0000,0.0019,0.3882,1,0.5892,0.3977,0.3153,27500,16127
7,Athens State University,Athens,AL,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0174,0.0057,0.0334,0.5517,1,0.4088,0.6296,0.6410,39000,18595
8,Auburn University at Montgomery,Montgomery,AL,0.0,0.0,0.0,0,486.0,509.0,0.0,...,0.0297,0.0397,0.0246,0.2853,1,0.4192,0.5803,0.2930,35000,21335
9,Auburn University,Auburn,AL,0.0,0.0,0.0,0,575.0,588.0,0.0,...,0.0000,0.0100,0.0140,0.0862,1,0.1610,0.3494,0.0415,45700,21831


### Problem 12
<span  style="color:green; font-size:16px">Which top 5 historically black colleges that have the highest white percentage?</span>